In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from pprint import pprint
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn import preprocessing
import pickle
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")

In [2]:
train_data = pd.read_csv('csv_data/train_data.csv')
train_data = train_data.round(2)
train_data

,Infant,Heart Rate,Time (sec),Respiratory Rate
0,1,164.84,184.32,127.66
1,1,163.04,187.29,21.16
2,1,163.04,188.78,46.73
3,1,164.84,188.89,46.73
4,1,163.04,189.46,88.24
...,...,...,...,...
118672,10,106.01,158928.38,93.75
118673,10,102.04,160485.40,93.75
118674,10,104.17,161948.56,93.75
118675,10,245.90,162241.24,93.75


In [3]:
X = train_data[['Time (sec)','Respiratory Rate']] 
y = train_data['Heart Rate']  

In [4]:
sc = preprocessing.StandardScaler()
X = sc.fit_transform(X)
y =  sc.fit_transform(y.values.reshape(-1, 1))

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [6]:
import random

# Select a subset of the data
subset_index = random.sample(range(0, len(X_train)), 1000)
X_train_subset = X_train[subset_index, :]
y_train_subset = y_train[subset_index]


In [7]:
from sklearn.svm import SVR
#create an SVR object
svr = SVR()

In [8]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {'C': [0.1, 1, 10], 'gamma': [0.1, 1, 10]}

# Create a GridSearchCV object
grid_search = GridSearchCV(svr, param_grid, cv=5)

# Train the model using the subset of the data
grid_search.fit(X_train_subset, y_train_subset)

# Print the best parameters
print(grid_search.best_params_)

{'C': 0.1, 'gamma': 1}


In [10]:
# Get the best parameters from the grid search
best_params = grid_search.best_params_

# Create an SVR model using the best parameters
svr_optimized = SVR(C=best_params['C'], gamma=best_params['gamma'])

# Train the optimized SVR model
svr_optimized.fit(X_train, y_train)

SVR(C=0.1, gamma=1)

In [13]:
y_pred = svr_optimized.predict(X_test)

In [14]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_squared_log_error, median_absolute_error

# calculate R-squared
r2 = r2_score(y_test, y_pred)

# calculate MSE
mse = mean_squared_error(y_test, y_pred)

# calculate RMSE
rmse = np.sqrt(mse)

#print the results
print("R-squared value: ", r2)
print("Mean Squared Error: ", mse)
print("Root Mean Squared Error: ", rmse)

R-squared value:  0.01920184888212617
Mean Squared Error:  0.9840853401713959
Root Mean Squared Error:  0.992010756076463


In [16]:
# Save the model
filename = 'saved_models/svr.sav'
pickle.dump(svr_optimized, open(filename, 'wb'))